In [7]:
import pandas as pd
from sklearn.linear_model import LinearRegression

data = pd.read_csv('Advertising.csv')
x = data.drop(['Unnamed: 0', 'sales'], axis=1)
y = data['sales']

In [9]:
model = LinearRegression().fit(x, y)

In [10]:
print(f'{model.coef_}, {model.intercept_}')

[ 0.04576465  0.18853002 -0.00103749], 2.9388893694594067


In [11]:
model.score(x, y)

0.8972106381789522

In [18]:
x_mat = np.concatenate([np.ones((len(x), 1)), x], axis=1)

In [19]:
x_mat.shape

(200, 4)

In [20]:
np.linalg.inv(x_mat.transpose(1, 0) @ x_mat) @ x_mat.transpose(1, 0) @ y

array([ 2.93888937e+00,  4.57646455e-02,  1.88530017e-01, -1.03749304e-03])

In [21]:
lhs = x_mat.transpose(1, 0) @ x_mat
rhs = x_mat.transpose(1, 0) @ y

In [22]:
lhs

array([[2.00000000e+02, 2.94085000e+04, 4.65280000e+03, 6.11080000e+03],
       [2.94085000e+04, 5.79111839e+06, 6.98061980e+05, 9.19625280e+05],
       [4.65280000e+03, 6.98061980e+05, 1.52107860e+05, 1.64946550e+05],
       [6.11080000e+03, 9.19625280e+05, 1.64946550e+05, 2.81096740e+05]])

In [23]:
rhs

array([  2804.5 , 482108.34,  74126.39,  90851.03])

In [26]:
def cramer(a, b):
    x = []
    d = np.linalg.det(a)
    for i in range(len(b)):
        a_temp = a.copy()
        a_temp[:, i] = b
        x.append(np.linalg.det(a_temp) / d)
    return x

In [27]:
cramer(lhs, rhs)

[2.9388893694594564,
 0.045764645455397684,
 0.1885300169182058,
 -0.0010374930424762781]

1. Loss function used in `LinearRegression`? Other possibilities?
2. Optimization method used in `LinearRegression` by default? Other possibilities?
3. Find analytical solution for coefficients and compare with those from `LinearRegression`.

## 1. What loss function is used by `LinearRegression`?

The `LinearRegression` class only supports the mean-squared error loss function. According to the documentation,

> `LinearRegression` fits a linear model with coefficients w = (w1, …, wp) to minimize the residual sum of squares between the observed targets in the dataset, and the targets predicted by the linear approximation.

It can, however, fit a linear model without an intercept ($\theta_0 = 0$, in our formulation) or with only nonnegative coefficients ($\theta_i \ge 0$, all $i$).

## 2. What optimization method is used by `LinearRegression`?

This was a little trickier to track down. Looking at the [source code](https://github.com/scikit-learn/scikit-learn/blob/7f9bad99d/sklearn/linear_model/_base.py#L534) for `LinearRegression`, it uses `scipy.linalg.lstsq` normally. If nonnegative coefficients are required, then it uses `scipy.optimize.nnls`. If the data is given as a sparse array, then it uses `scipy.sparse.linalg.lsqr`.

The `scipy.linalg.lstsq` function uses one of three LAPACK functions internally: `gelsd` (default), `gelsy`, or `gelss`. The `gelss` method uses the singular value decomposition of the coefficient matrix $A$ to solve $Ax = b$. The `gelsy` method uses an orthogonal factorization method. The `gelsd` method also uses singular value decomposition, but computes it in a (presumably) faster way than `gelss`.